![wcar](static/wcar.png)
# Over Expression Library: NMT

    Wellcome Centre for Anti-Infectives Research
    School of Life Sciences, University of Dundee

# Analysis start

In [ ]:
#reload when modified
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#load labrary
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from IPython.display import Image

In [ ]:
import sys
sys.path.insert(0, "mylib")

In [ ]:
from plot_region_coverage import plot_region
from plot_region_coverage import get_field
from plot_region_coverage import gff_to_pandas

In [ ]:
#import itables.interactive
from itables import show

## Helper functions

In [ ]:
_EXPERIMENT = '{_EXPERIMENT}'
_FASTQ_HEADER = '{_FASTQ_HEADER}'
_GFF = os.path.join('genomes','{genome}','{genome}.gff')

## Read GFF file

In [ ]:
gff = gff_to_pandas(_GFF)
gff.set_index('gene_id',inplace=True)
#print(gff.shape)
gff.head()

## Read Peaks

In [ ]:
path_to_peaks = os.path.join(_EXPERIMENT,'macs2_'+_FASTQ_HEADER,'NA_peaks.xls')
peaks = pd.read_csv(path_to_peaks, sep='\t', index_col=None, comment='#')
peaks.sort_values('pileup',ascending=False,inplace=True)
peaks.rename({'chr':'region_chr','start':'region_start','end':'region_end',
             'length':'region_length','pileup':'region_pileup',
              'name':'region_name'},axis=1,inplace=True)
peaks.drop(['-log10(pvalue)','fold_enrichment','-log10(qvalue)'],inplace=True,axis=1)
print(peaks.shape)
peaks.head(10)
#filter out small count peaks
peaks = peaks[peaks['region_pileup']>peaks['region_pileup'].mean()]
peaks = peaks.head(20)
print(peaks.shape)

## Read Gene Counts

In [ ]:
path_to_counts = os.path.join(_EXPERIMENT,'counts.txt')
counts = pd.read_csv(path_to_counts, sep='\t', index_col=None, comment='#')
print(counts.head())
counts.columns = ['gene_id', 'cchr', 'cstart', 'cend', 'cstrand', 'cgenelen',
                  'counts','counts_ff','counts_fr','counts_rf','counts_rr']
counts.set_index('gene_id',inplace=True)
print(counts.shape)
counts.head()

## Merge counts and GFF file

In [ ]:
genes = counts.join(gff)
print(genes.shape)
genes.head(10)
genes.reset_index(drop=False,inplace=True)
genes.head()

## Find region of interest

In [ ]:
def test(genes, peaks, gene):
    gene_start = genes.loc[gene]['gene_start']
    gene_end = genes.loc[gene]['gene_end']
    gene_chr = genes.loc[gene]['gene_chr']
    temp_res = []
    for peak in peaks.index.values:
        peak_start = peaks.loc[peak]['region_start']
        peak_end = peaks.loc[peak]['region_end']
        peak_chr = peaks.loc[peak]['region_chr'] 
        region_name = peaks.loc[peak]['region_name']
        region_count = peaks.loc[peak]['region_pileup']
        if (gene_chr == peak_chr):
            if (peak_start-2000 <= gene_start <= peak_end+2000) or (peak_start-2000 <= gene_end <= peak_end+2000):
                temp_res.append(region_name)
    if len(temp_res)>0:
        return (gene,';'.join(temp_res))
    else:
        return (gene,'NONE')

In [ ]:
import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())
#pool = mp.Pool(mp.cpu_count())
pool = mp.Pool(8)

In [ ]:
results = [pool.apply_async(test, args=(genes, peaks, gene)) for gene in genes.index.values]
pool.close()

In [ ]:
%%time
results = [n.get() for n in results]

In [ ]:
#%%time
#results = [n.get() for n in results]

In [ ]:
temp = pd.DataFrame()
temp['genes']=[n[0] for n in results]
temp['region_name']=[n[1] for n in results]
temp.set_index('genes',inplace=True)
temp.head()

In [ ]:
genes = genes.join(temp)
genes.head()

## Add region to results

this is the full result table

In [ ]:
genes = pd.merge(genes, peaks, right_on='region_name', left_on='region_name', how='left')
print(genes.shape)
#genes.head(10)

In [ ]:
genes.sort_values(['counts','region_pileup']).dropna().tail(5)

In [ ]:
genes.to_csv(_EXPERIMENT+'/res.csv')

## Plot Genome Coverage

In [ ]:
Image(filename=os.path.join(_EXPERIMENT,_FASTQ_HEADER+'coverage_d.png')) 